In [74]:
#Documentation for yfinance https://pypi.org/project/yfinance/

import yfinance as yf
from sqlalchemy import create_engine
import pandas as pd
import json

# Beginning of Stage 2 - Read stock price database tables from PostgreSQL and save as JSON files. One file for each stock.  
JSON File format:  
[[Date, Open, High, Low, Close, Volume], [Date, Open, High, Low, Close, Volume]...[Date, Open, High, Low, Close, Volume]]


In [75]:
#Connect to PostgreSQL

host = "localhost"
user = "postgres"
port = "5432"
passwd = "hawkeyes"
db = "Proj2_stock_data"

engine = create_engine(f'postgresql://{user}:{passwd}@{host}:{port}/{db}')
connection = engine.connect()


In [76]:
#Function to create a dataframe from a PostgreSQL query
def create_pandas_table(sql_query, database = connection):
    table = pd.read_sql_query(sql_query, database)
    return table

In [77]:
#Query each stock's PostgreSQL table and convert to dataframe

gspc_df = create_pandas_table('SELECT * FROM public."GSPC_hist"')
n225_df = create_pandas_table('SELECT * FROM public."N225_hist"')
hsi_df = create_pandas_table('SELECT * FROM public."HSI_hist"')
ssec_df = create_pandas_table('SELECT * FROM public."SSEC_hist"')

connection.close()

In [78]:
gspc_df = gspc_df.drop(['Dividends', 'Stock Splits'], axis=1)
gspc_df['Date'] = gspc_df['Date'].astype(str)
gspc_lst = gspc_df.values.tolist()
with open('gspc_hist.json', 'w') as F:
    F.write(json.dumps(gspc_lst))

In [79]:
hsi_df = hsi_df.drop(['Dividends', 'Stock Splits'], axis=1)
hsi_df['Date'] = hsi_df['Date'].astype(str)
hsi_lst = hsi_df.values.tolist()
with open('hsi_hist.json', 'w') as F:
    F.write(json.dumps(hsi_lst))

In [80]:
n225_df = n225_df.drop(['Dividends', 'Stock Splits'], axis=1)
n225_df['Date'] = n225_df['Date'].astype(str)
n225_lst = n225_df.values.tolist()
with open('n225_hist.json', 'w') as F:
    F.write(json.dumps(n225_lst))

In [81]:
ssec_df = ssec_df.drop(['Dividends', 'Stock Splits'], axis=1)
ssec_df['Date'] = ssec_df['Date'].astype(str)
ssec_lst = ssec_df.values.tolist()
with open('ssec_hist.json', 'w') as F:
    F.write(json.dumps(ssec_lst))

# End of Stage 2 - JSON files can be pushed to endpoint URL's for each stock using Flask.  
For example:  
http://project2/gspc
http://project2/n225
http://project2/hsi
http://project2/ssec . 
  
This data can then be pulled into Javascript using d3.json and then used for table data.
